In [ ]:
import numpy as np
import pandas as pd

### **Step 0. Formulate your research objective and understand your data**

**Research Objective:** Preprocess data directly obtained from the 3000 Rice Genomes Project, so that it is in simple matrix form. Our processing will transform it so that markers will be in rows and varieties in columns. Also, missing data will be imputed simply with the most common allele at each marker site across the population (bad algorithm, but easy for demonstration!)  
**Problem Type:** Sequence marker pre-processing  
**Study Period:** 2017-2021   
**Data:**

- Marker data across approximately 3000 rice varieties

 ### Step 1. Reading, exploring, and cleaning the vcf file for chromosome 1
 Skipping the first 6 rows since they are not in tabular format

In [ ]:
df = pd.read_csv("/opt/hpc4ag/data/3k-core-v7-chr1/chr1.vcf", sep="\t", skiprows=6)

In [ ]:
df.head()

In [ ]:
df.shape

Reading in the additive matrix file generated using plink
This is a space delimited file
Here, the rows indicate individuals genotyped, and columns are the loci in addition to some 
initial columns identifying if it was a mother or father in a cross or a sex chromosome or if there is an associated phenotype 


In [ ]:
df2 = pd.read_csv("/opt/hpc4ag/data/3k-core-v7-chr1/chr1_A.raw", sep='\\s+')

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_3225362/1003293286.py:1: SyntaxWarning: invalid escape sequence '\s'
  df2 = pd.read_csv("/home/gems_learning/shared/hpc4ag/3k-core-v7-chr1/chr1_A.raw", sep = '\s+')


In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.tail()

Doing some exploratory analysis on some of the columns

In [ ]:
df2["FID"].nunique()

In [ ]:
df2["PAT"].unique().tolist()

In [ ]:
df2["MAT"].unique().tolist()

In [ ]:
df2["SEX"].unique().tolist()

In [ ]:
df2["PHENOTYPE"].unique().tolist()

Seeing as columns MAT PAT SEX and PHENOTYPE have no data of interest as such, we can drop them before proceeding

In [ ]:
labels = ["FID", "PAT", "MAT", "SEX", "PHENOTYPE"]
df2 = df2.drop(columns=labels, inplace = False)

In [ ]:
df2.head()

In [ ]:
df2.shape

We can make the individual ID column into an index so it becomes easier to transpose the matrix if required

In [ ]:
df2 = df2.set_index('IID')

In [ ]:
df2.head()

In [ ]:
df2.head()

In [ ]:
df2.isnull().any(axis=1).sum()

## Impute missing values
Because a large number of missing values are present, we need to impute the values instead of simply dropping those rows or loci. This could be time-consuming, so lets see how it scales!
### timing for 100 variety subset

In [ ]:
%%time
cols = df100v.columns.tolist()
df100v[cols] = df100v[cols].fillna(df100v.mode().iloc[0])

CPU times: user 38.3 s, sys: 214 ms, total: 38.6 s
Wall time: 38.6 s


### timing for 200 variety subset

In [23]:
print("Initial shape: "+str(df2.shape))
df200v = df2.iloc[0:200, :].copy()
print("Subset shape: "+str(df200v.shape))

Initial shape: (3024, 42466)
Subset shape: (200, 42466)


In [24]:
%%time
cols = df200v.columns.tolist()
df200v[cols] = df200v[cols].fillna(df200v.mode().iloc[0])

CPU times: user 39.6 s, sys: 359 ms, total: 39.9 s
Wall time: 39.9 s


### timing for 400 variety subset

In [25]:
print("Initial shape: "+str(df2.shape))
df400v = df2.iloc[0:400, :].copy()
print("Subset shape: "+str(df400v.shape))

Initial shape: (3024, 42466)
Subset shape: (400, 42466)


In [26]:
%%time
cols = df400v.columns.tolist()
df400v[cols] = df400v[cols].fillna(df400v.mode().iloc[0])

CPU times: user 41.5 s, sys: 269 ms, total: 41.7 s
Wall time: 41.9 s


### timing for 800 variety subset

In [27]:
print("Initial shape: "+str(df2.shape))
df800v = df2.iloc[0:800, :].copy()
print("Subset shape: "+str(df800v.shape))

Initial shape: (3024, 42466)
Subset shape: (800, 42466)


In [28]:
%%time
cols = df800v.columns.tolist()
df800v[cols] = df800v[cols].fillna(df800v.mode().iloc[0])

CPU times: user 43 s, sys: 369 ms, total: 43.4 s
Wall time: 43.7 s


## Conclusion of our variety scaling test
OK, we clearly see the execution time is invariant with respect to the number of varieties we've included!! So we can estimate that
3024 varieties should also take ~40 sec.

But what if we had fewer markers. Logic would suggest that will be different. Recall we are doing a dumb imputation where we are just looking across the population of 3024 individuals and setting NULL entries to the most frequent allelic state of that marker. Doing the mode calc across 3024 individuals isn't a lot harder than across 100. But we are repeating this over 42,000 times!! If we only had 100 markers, we'd do a mode calculation 100 times not 42,000 times. Let's see.

### timing for 1000 marker subset

In [29]:
print("Initial shape: "+str(df2.shape))
df1000m = df2.iloc[:, 0:1000].copy()
print("Subset shape: "+str(df1000m.shape))

Initial shape: (3024, 42466)
Subset shape: (3024, 1000)


In [30]:
%%time
cols = df1000m.columns.tolist()
df1000m[cols] = df1000m[cols].fillna(df1000m.mode().iloc[0])

CPU times: user 368 ms, sys: 21.7 ms, total: 390 ms
Wall time: 387 ms


### timing for 2000 marker subset

In [31]:
print("Initial shape: "+str(df2.shape))
df2000m = df2.iloc[:, 0:2000].copy()
print("Subset shape: "+str(df2000m.shape))

Initial shape: (3024, 42466)
Subset shape: (3024, 2000)


In [32]:
%%time
cols = df2000m.columns.tolist()
df2000m[cols] = df2000m[cols].fillna(df2000m.mode().iloc[0])

CPU times: user 870 ms, sys: 26.8 ms, total: 897 ms
Wall time: 893 ms


### timing for 4000 marker subset

In [33]:
print("Initial shape: "+str(df2.shape))
df4000m = df2.iloc[:, 0:4000].copy()
print("Subset shape: "+str(df4000m.shape))

Initial shape: (3024, 42466)
Subset shape: (3024, 4000)


In [34]:
%%time
cols = df4000m.columns.tolist()
df4000m[cols] = df4000m[cols].fillna(df4000m.mode().iloc[0])

CPU times: user 1.85 s, sys: 41.6 ms, total: 1.89 s
Wall time: 1.89 s


### timing for 8000 marker subset

In [35]:
print("Initial shape: "+str(df2.shape))
df8000m = df2.iloc[:, 0:8000].copy()
print("Subset shape: "+str(df8000m.shape))

Initial shape: (3024, 42466)
Subset shape: (3024, 8000)


In [36]:
%%time
cols = df8000m.columns.tolist()
df8000m[cols] = df8000m[cols].fillna(df8000m.mode().iloc[0])

CPU times: user 4.64 s, sys: 97.3 ms, total: 4.73 s
Wall time: 4.74 s


## Conclusion of our marker scaling test
So that's kind of what we expected. Clear evidence of complexity worse than linear (O(N)). We can clearly see this is requiring more than ~0.6 s for every 1000 markers. So it should take > 5 s / 8000 * 42466 = ~27 sec. That's an underestimate since it assumes linearity, and it clearly is taking more and more time with each doubling. But not a lot more. No need to plot the timings and model the complexity... unless you want to!

In [37]:
%%time
cols = df2.columns.tolist()
df2[cols] = df2[cols].fillna(df2.mode().iloc[0])

CPU times: user 42.4 s, sys: 1.22 s, total: 43.6 s
Wall time: 43.8 s


In [ ]:
df2.dtypes

In [ ]:
df2.head()

In [ ]:
df2.tail()

In [ ]:
df2["1178_T"].value_counts()

In [ ]:
# Transposing the matrix so we have the loci as the rows and individuals as the columns
df2 = df2.T

In [ ]:
df2.head()

In [ ]:
# Counting the number of rows that have null values
df2.shape[0] - df2.dropna().shape[0]

In [ ]:
df2.shape

In [ ]:
# Saving this as a csv file for further processing
df2.to_csv("chr1_A_matrix.csv")